Link: https://www.kaggle.com/datasets/spscientist/students-performance-in-exams?resource=download

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('DataSets\Original_data_with_more_rows.csv')
df.head()

,Id,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,MathScore,ReadingScore,WritingScore
0,0,female,group B,bachelor's degree,standard,none,72,72,74
1,1,female,group C,some college,standard,completed,69,90,88
2,2,female,group B,master's degree,standard,none,90,95,93
3,3,male,group A,associate's degree,free/reduced,none,47,57,44
4,4,male,group C,some college,standard,none,76,78,75


turn categorical attributes into numerical attributes


In [2]:
df = df[[c for c in df if c not in ['MathScore']] + ['MathScore']]

feats_to_drop = ['Id']
df = df.drop(feats_to_drop, axis=1)

ord_education = ['some high school', 'high school', 'some college', "associate's degree", "bachelor's degree", "master's degree"]
for num, level in enumerate(ord_education):
    df.replace(to_replace=level, value=num, inplace=True)

feats_to_one_hot_encode = ['Gender', 'EthnicGroup', 'LunchType', 'TestPrep']
df = pd.get_dummies(df, columns=feats_to_one_hot_encode)

feats_to_standardize = ['ParentEduc', 'ReadingScore', 'WritingScore']
for feat in feats_to_standardize:
    df[feat] = (df[feat] - df[feat].mean()) / df[feat].std()

features = df.iloc[:, :df.shape[1] - 1].values
labels = df.loc[:, 'MathScore'].values

In [3]:
def generateRandom(data, features, labels):
  perm_idx = np.random.permutation(data.shape[0])
  vali_num = int(data.shape[0] * 0.2)
  vali_idx = perm_idx[:vali_num]
  train_idx = perm_idx[vali_num:]
  train_features = features[train_idx, :]
  train_labels = labels[train_idx]
  vali_features = features[vali_idx, :]
  vali_labels = labels[vali_idx]
  return train_features, train_labels, vali_features, vali_labels

train_features, train_labels, vali_features, vali_labels = generateRandom(df, features, labels)
print(train_features.shape)
print(train_labels.shape)
print(vali_features.shape)
print(vali_labels.shape)

(24513, 14)
(24513,)
(6128, 14)
(6128,)


In [4]:
def KNN(train_features, train_labels, test_features, k=10):
    vali_pred = []
    for i in range(test_features.shape[0]):
        x = test_features[i, :]  
        distances = np.sqrt(np.sum((x - train_features) ** 2, axis=1))
        topk_idx = np.argpartition(distances, k)[:k]
        topk_labels = list(train_labels[topk_idx])
        pred = max(topk_labels, key=topk_labels.count)
        vali_pred.append(pred)
    return np.array(vali_pred)
for k_tuner in range(5, 16):
    pred = KNN(train_features, train_labels, vali_features, k=k_tuner)

    count = 0
    margin = 0
    for i in range(pred.size):
        if vali_labels[i] - margin <= pred[i] and pred[i] <= vali_labels[i] + margin:
            count += 1

    print("K: {}, Accuracy: {}".format(k_tuner, str(count / vali_features.shape[0])))

K: 5, Accuracy: 0.9471279373368147
K: 6, Accuracy: 0.9472911227154047
K: 7, Accuracy: 0.943701044386423
K: 8, Accuracy: 0.9427219321148825
K: 9, Accuracy: 0.9425587467362925
K: 10, Accuracy: 0.9433746736292428
K: 11, Accuracy: 0.9441906005221932
K: 12, Accuracy: 0.9446801566579635
K: 13, Accuracy: 0.9490861618798956
K: 14, Accuracy: 0.9510443864229765
K: 15, Accuracy: 0.952023498694517
